In [3]:
##パッケージの読み込み
import math
import os
import pickle
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
kfold = StratifiedKFold(n_splits=4) #4層化クロスバリデーション

##データの読み込み
df2 = pd.read_csv('dataset_test_201709.csv')#.drop("Unnamed: 0",axis=1)#テストデータ
df = pd.read_csv('201609_201708_dataset.csv', encoding='utf-8').drop("Unnamed: 0",axis=1)

##trainデータの部品のデータフレームを作る
db = pd.DataFrame(df["部品品番"].value_counts())
db = db.reset_index().rename(columns={"index":"buhin","部品品番":"freq"})
db = db[db["freq"]>=4]
df.head()
#データを見て余計な特徴量があったら削除
df = df.drop("修理依頼内容",axis=1)

##特徴量の数をテストデータに合わせる
df = df[df.columns[df.columns.isin(df2.columns)]]
df2 = df2[df2.columns[df2.columns.isin(df.columns)]]
#部品のデータフレームを作る　buhin2は頻出部品のうちテストデータに含まれているもの
buhin = db.loc[:,"buhin"]
buhin = sorted(list(set(buhin)))
#テストデータ
db2 =df2[(df2["部品品番"].isin(db["buhin"]))]
buhin2 = db2.loc[:,"部品品番"]
buhin2 = sorted(list(set(buhin2)))

##機械学習用にデータを整形
#各モデルと予測結果の精度を格納するdataframe
DNN_train_validation = pd.DataFrame(0, columns=["acc"], index=buhin)
RF_train_validation = pd.DataFrame(0, columns=["acc"], index=buhin)
SVM_train_validation = pd.DataFrame(0, columns=["acc"], index=buhin)
LR_train_validation = pd.DataFrame(0, columns=["acc"], index=buhin)
mean_ensemble_train_validation = pd.DataFrame(0, columns=["acc"], index=buhin)
ensemble_train_validation = pd.DataFrame(0, columns=["acc"], index=buhin)
#accuracy
DNN_test = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
RF_test = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
SVM_test = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
LR_test = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
mean_ensemble_test = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
ensemble_test = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
#precsion
DNN_test2 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
RF_test2 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
SVM_test2 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
LR_test2 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
mean_ensemble_test2 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
ensemble_test2 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
#recall
DNN_test3 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
RF_test3 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
SVM_test3 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
LR_test3 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
mean_ensemble_test3 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
ensemble_test3 = pd.DataFrame(0, columns=["acc1","acc2","acc3","acc4","acc5","acc6","acc7","acc8","acc9","acc10"],index=buhin2)
#モデルを格納するdataframe #ランダムフォレストだけdictionary
DNN = pd.DataFrame(0, columns=["model"], index=buhin2)
RF = dict()
SVM = pd.DataFrame(0, columns=["model"], index=buhin2)
LR = pd.DataFrame(0, columns=["model"], index=buhin2)

CM = dict() #混同行列を格納する辞書 #一発修理の混同行列だけ，最後に別に用意した
s=0 #乱数シードの番号初期化
repeat=0 #for文の途中経過を見る。
names = df.columns #データの列名のlist


sc = StandardScaler()
DF1 = df.iloc[:,0:3]
DF2 = df.iloc[:,3:len(df.columns)]
DF2 = pd.DataFrame(sc.fit_transform(DF2))
DF2.columns = names[3:len(df.columns)]
df = pd.concat([DF1,DF2],axis=1)

#testデータにも適用（fit_transformは使ってはいけない）
DF1 = df2.iloc[:,0:3]
DF2 = df2.iloc[:,3:len(df2.columns)]
DF2 = pd.DataFrame(sc.transform(DF2))
DF2.columns = names[3:len(df2.columns)]
df2 = pd.concat([DF1,DF2],axis=1)

del DF1,DF2 #不要なもの削除

round = lambda x:(x*2+1)//2 #四捨五入する関数。intとは微妙に違う。

##学習させる
##############################################################################
#モデルごとの学習時間を格納するリスト
all_time_DNN_train = np.zeros(len(buhin))
all_time_RF_train = np.zeros(len(buhin))
all_time_SVM_train = np.zeros(len(buhin))
all_time_LR_train = np.zeros(len(buhin))
repeat=0

##############################################################################
#部品毎のモデルを作る
for i in tqdm(buhin2):#頻出300部品のループ
        repeat += 1
        ##########################################################################
        t1 = df.loc[df['部品品番'] == i] #train_validationから、ある頻出部品iだけを抽出
        t1["t"] = 1 #正解ラベル1の列tを作る
        X1 = t1.iloc[:,3:len(t1.columns)-1] #正解のデータから、特徴量部分だけを抽出
        y1 = t1.loc[:,"t"] #正解のデータから、正解ラベル部分だけを抽出

        key_diff = set(df.受付NO).difference(t1.受付NO) #train_validationの受付Noの中で、正解ラベルが含まれていない受付NOを抽出
        where_diff = df.受付NO.isin(key_diff) #その位置を特定
        t0 = df.loc[where_diff,:] #train_validationから不正解のデータを抽出
        t0["t"] = 0 #不正解ラベル0の列tを作る

    #     np.random.seed(s) #乱数固定
    #     t0 = t0.reindex(np.random.permutation(t0.index)) #行をシャッフルしてindexを付けなおす
        t0 = t0.sample(frac=1, random_state=repeat) 

        t0 = t0.drop_duplicates(['受付NO']) #受付NOが重複しないように行を消す

        t0 = t0.iloc[0:len(t1.index),:] #正解ラベルと同じ行数だけ不正解ラベルを取ってくる
        X0 = t0.iloc[:,3:len(t0.columns)-1] #不正解のデータから、特徴量部分だけを抽出
        y0 = t0.loc[:,"t"] #不正解のデータから、正解ラベル部分だけを抽出

        X = pd.concat([X1,X0]) #train_validationの特徴量
        y = pd.concat([y1,y0]) #train_validationの正解ラベル
        X = X.reindex()
        y = y.reindex()

        del X1,X0,y1,y0 #不要なオブジェクトを消す

        ############################################################################
        #グリッドサーチとクロスバリデーションで、validationの精度が最大になるパラメータを探す
        #正解と不正解の比が同じなのでaccuracyで見る
        #DNN
        start_time = time.time()
        best_score = 0 #ベストスコアを初期化
        for L1 in [100,200,300]:
            for L2 in [100,200,300]:
                model1 = MLPClassifier(solver="adam", random_state=0, activation="relu", max_iter=100, alpha=1, hidden_layer_sizes=[L1,L2])
                scores = cross_val_score(model1, X, y, cv=kfold)
                score = np.nanmean(scores)
                if score > best_score:
                    best_score = score
                    best_L1 = L1
                    best_L2 = L2        
        model1 = MLPClassifier(solver="adam", random_state=0, activation="relu", max_iter=100, alpha=1, hidden_layer_sizes=[best_L1,best_L2])
        clf1 = model1.fit(X,y)
        end_time = time.time()
        all_time_DNN_train[repeat-1] = end_time - start_time

        #RF
        start_time = time.time()
        best_score = 0
        for n in [int(round(math.sqrt(len(X.columns)))), int(round(math.sqrt(len(X.columns))))+5, int(round(math.sqrt(len(X.columns))))+10]:
            model2 = RandomForestClassifier(n_estimators=50, max_features=n, random_state=0)
            scores = cross_val_score(model2, X, y, cv=kfold)
            score = np.nanmean(scores)
            if score > best_score:
                best_score = score
                best_n = n   
        model2 = RandomForestClassifier(n_estimators=50, max_features=best_n, random_state=0)
        clf2 = model2.fit(X, y)
        end_time = time.time()
        all_time_RF_train[repeat-1] = end_time - start_time

        #SVM
        start_time = time.time()
        best_score = 0
        for c in [0.001,0.01,0.1,1,10,100]:
            model3 = CalibratedClassifierCV(LinearSVC(C=c))
            scores = cross_val_score(model3, X, y, cv=kfold)
            score = np.nanmean(scores)
            if score > best_score:
                best_score = score
                best_c = c        
        model3 = CalibratedClassifierCV(LinearSVC(C=best_c))
        clf3 = model3.fit(X,y)        
        end_time = time.time()
        all_time_SVM_train[repeat-1] = end_time - start_time        

        #LR
        start_time = time.time()
        best_score = 0
        for c in [0.001,0.01,0.1,1,10,100]:
            model4 = LogisticRegression(C=c)
            scores = cross_val_score(model4, X, y, cv=kfold)
            score = np.nanmean(scores)
            if score > best_score:
                best_score = score
                best_c = c        
        model4 = LogisticRegression(C=best_c)
        clf4 = model4.fit(X,y)             
        end_time = time.time()
        all_time_LR_train[repeat-1] = end_time - start_time

        pre1 = clf1.predict_proba(X)[:,1]
        pre2 = clf2.predict_proba(X)[:,1]
        pre3 = clf3.predict_proba(X)[:,1]
        pre4 = clf4.predict_proba(X)[:,1]
        pre_mat = pd.DataFrame({"DNN":pre1,"RF":pre2,"SVM":pre3,"LR":pre4},
                               index=np.r_[0:len(pre1)])
        pre_mat["count"] = (pre_mat >= 0.5).sum(axis=1) 
        pre_mat["predict"] = 0
        pre_mat["predict_mean"] = round((pre1 + pre2 + pre3 + pre4)/4).astype(int)#平均値を四捨五入する→0.5を超えれば1になる
        for k in list(pre_mat.index):
            if pre_mat.loc[k,"count"] == 2:#それぞれの識別器の結果が2対2になったとき
                pre_mat.loc[k,"predict"] = round(np.median(pre_mat.loc[k,"DNN":"LR"])).astype(int) #中央値が0.5を超えれば1になる
            elif pre_mat.loc[k,"count"] > 2:
                pre_mat.loc[k,"predict"] = 1
        CM["buhin_train_validation_" + str(i)] = confusion_matrix(y, pre_mat.loc[:,"predict"]) #部品毎のモデルのtrain_validationの多数決モデルの混同行列
        CM["buhin_train_validation_mean_" + str(i)] = confusion_matrix(y, pre_mat.loc[:,"predict_mean"]) #部品毎のモデルのtrain_validationの平均モデルの混同行列

        #精度を表示したり、保存したり色々（ランダムフォレストだけdataframeに保存できないのdictに格納）
        DNN_train_validation.loc[i,"acc"] = clf1.score(X,y)
        RF_train_validation.loc[i,"acc"] = clf2.score(X,y)
        SVM_train_validation.loc[i,"acc"] = clf3.score(X,y)
        LR_train_validation.loc[i,"acc"] = clf4.score(X,y)
        mean_ensemble_train_validation.loc[i,"acc"] = (CM["buhin_train_validation_mean_" + str(i)][0,0]+CM["buhin_train_validation_mean_" + str(i)][1,1])/np.sum(CM["buhin_train_validation_mean_" + str(i)]) #部品毎のモデルのtrain_validationの平均モデルのaccuracy
        ensemble_train_validation.loc[i,"acc"] = (CM["buhin_train_validation_" + str(i)][0,0]+CM["buhin_train_validation_" + str(i)][1,1]) / np.sum(CM["buhin_train_validation_" + str(i)]) #部品毎のモデルのtrain_validationの多数決モデルのaccuracy
        print("DNN_train_validation : " + str(DNN_train_validation.loc[i,"acc"]))
        print("RF_train_validation : " + str(RF_train_validation.loc[i,"acc"]))
        print("SVM_train_validation : " + str(SVM_train_validation.loc[i,"acc"]))
        print("LR_train_validation : " + str(LR_train_validation.loc[i,"acc"]))
        print("mean_ensemble_train_validation : " + str(mean_ensemble_train_validation.loc[i,"acc"]))
        print("ensemble_train_validation : " + str(ensemble_train_validation.loc[i,"acc"]))

        #データフレームに学習済みモデルを格納する
        DNN.loc[i,"model"] = clf1
        RF[i] = clf2
        SVM.loc[i,"model"] = clf3
        LR.loc[i,"model"] = clf4

        print("repeat " + str(repeat) + " end")
        
##モデルを保存する
with open('DNN_201609-201708.pickle', mode='wb') as DNN_model:
    pickle.dump(DNN, DNN_model)
with open('RF_201609-201708.pickle', mode='wb') as RF_model:
    pickle.dump(RF, RF_model)
with open('SVM_201609-201708.pickle', mode='wb') as SVM_model:
    pickle.dump(SVM, SVM_model)
with open('LR_201609-201708.pickle', mode='wb') as LR_model:
    pickle.dump(LR, LR_model)
    
##モデルを読み込む
with open('DNN_201609-201708.pickle',mode='rb')as DNN_model:
    DNN = pickle.load(DNN_model)
with open('RF_201609-201708.pickle',mode='rb')as RF_model:
    RF = pickle.load(RF_model)
with open('SVM_201609-201708.pickle',mode='rb')as SVM_model:
    SVM = pickle.load(SVM_model)
with open('LR_201609-201708.pickle',mode='rb')as LR_model:
    LR = pickle.load(LR_model)
#各モデルの精度を格納してcsvに書き出し
DNN_train_validation.to_csv('DNN_train_validation_201609-201708.csv')
RF_train_validation.to_csv('RF_train_validation_201609-201708.csv')
SVM_train_validation.to_csv('SVM_train_validation_201609-201708.csv')
LR_train_validation.to_csv('LR_train_validation_201609-201708.csv')
#テストをするためにcolumn名を変える
db = db.rename(columns={'buhin':'部品品番'})

##保存したモデルを使ってテストする
from sklearn.metrics import precision_score,recall_score

for i in range(len(buhin2)):#
    for j in range(0,10): #10テーブルのループ
        if DNN.reset_index()['index'][i] in buhin2: #testに含まれる頻出部品ならば予測を行う
            t1 = df2.loc[df2['部品品番'] == DNN.reset_index()['index'][i]] #testから、ある頻出部品iだけを抽出
            t1["t"] = 1 #正解ラベル1の列tを作る
            X1 = t1.iloc[:,3:len(t1.columns)-1] #正解のデータから、特徴量部分だけを抽出
            y1 = t1.loc[:,"t"] #正解のデータから、正解ラベル部分だけを抽出

            key_diff = set(df2.受付NO).difference(t1.受付NO) #testの受付Noの中で、正解ラベルが含まれていない受付NOを抽出
            where_diff = df2.受付NO.isin(key_diff) #その位置を特定
            t0 = df2.loc[where_diff,:] #testから不正解のデータを抽出
            t0["t"] = 0 #正解ラベル0の列tを作る

            
#             t0 = t0.reindex(np.random.permutation(t0.index)) #行をシャッフルしてindexを付けなおす
            t0 = t0.sample(frac=1, random_state=j) #行をシャッフルしてindexを付けなおす
            t0 = t0.drop_duplicates(['受付NO']) #受付NOが重複しないように行を消す

            t0 = t0.iloc[0:len(t1.index),:] #正解ラベルと同じ行数だけ不正解ラベルを取ってくる
            X0 = t0.iloc[:,3:len(t0.columns)-1] #不正解のデータから、特徴量部分だけを抽出
            y0 = t0.iloc[:,len(t0.columns)-1] #不正解のデータから、正解ラベル部分だけを抽出

            X = pd.concat([X1,X0]) #testの特徴量部分
            y = pd.concat([y1,y0]) #testの正解ラベル部分

            #各モデルで1になる確率を予測
            pre1 = DNN.reset_index()['model'][i].predict_proba(X)[:,1]
            pre2 = list(RF.values())[i].predict_proba(X)[:,1]
            pre3 = SVM.reset_index()['model'][i].predict_proba(X)[:,1]
            pre4 = LR.reset_index()['model'][i].predict_proba(X)[:,1]

            print(pre1)
            pre_mat = pd.DataFrame({"DNN":pre1,"RF":pre2,"SVM":pre3,"LR":pre4},
                                   index=np.r_[0:len(pre1)])
            pre_mat["count"] = (pre_mat >= 0.5).sum(axis=1)
            pre_mat["predict"] = 0
           
            pre_mat["predict_mean"] = np.round((pre1 + pre2 + pre3 + pre4)/4).astype(int)
            for k in list(pre_mat.index):
                if pre_mat.loc[k,"count"] == 2:
                    pre_mat.loc[k,"predict"] = round(np.median(pre_mat.loc[k,"DNN":"LR"])).astype(int)
                elif pre_mat.loc[k,"count"] > 2:
                    pre_mat.loc[k,"predict"] = 1
            CM["buhin_test_" + str(i)] = confusion_matrix(y, pre_mat.loc[:,"predict"]) #testの部品毎のモデルの多数決モデルの混同行列
            CM["buhin_test_mean_" + str(i)] = confusion_matrix(y, pre_mat.loc[:,"predict_mean"]) #testの部品毎のモデルの平均モデルの混同行列
            #accuracy
            mean_ensemble_test.ix[i,j] = (CM["buhin_test_mean_" + str(i)][0,0]+CM["buhin_test_mean_" + str(i)][1,1])/np.sum(CM["buhin_test_mean_" + str(i)]) #testの部品毎のモデルの平均モデルのaccuracy
            ensemble_test.ix[i,j] = (CM["buhin_test_" + str(i)][0,0]+CM["buhin_test_" + str(i)][1,1])/np.sum(CM["buhin_test_" + str(i)]) #testの部品毎のモデルの多数決モデルのaccuracy
            #precsion
            mean_ensemble_test2.ix[i,j] = CM["buhin_test_mean_" + str(i)][0,0]/(CM["buhin_test_mean_" + str(i)][0,0]+ CM["buhin_test_mean_" + str(i)][0,1])#testの部品毎のモデルの平均モデルのaccuracy
            ensemble_test2.ix[i,j] = CM["buhin_test_" + str(i)][0,0]/(CM["buhin_test_" + str(i)][0,0]+CM["buhin_test_" + str(i)][0,1]) #testの部品毎のモデルの多数決モデルのprecision
            #recall
            mean_ensemble_test3.ix[i,j] = CM["buhin_test_mean_" + str(i)][0,0]/(CM["buhin_test_mean_" + str(i)][0,0]+ CM["buhin_test_mean_" + str(i)][1,0])#testの部品毎のモデルの平均モデルのaccuracy
            ensemble_test3.ix[i,j] =  CM["buhin_test_" + str(i)][0,0]/(CM["buhin_test_" + str(i)][0,0]+CM["buhin_test_" + str(i)][1,0]) #testの部品毎のモデルの多数決モデルのaccuracy
            
            DNN_pred = DNN.reset_index()['model'][i].predict(X)
            RF_pred = list(RF.values())[i].predict(X)
            SVM_pred = SVM.reset_index()['model'][i].predict(X)
            LR_pred = LR.reset_index()['model'][i].predict(X)
            #accuracy
            DNN_test.ix[i,j] = DNN.reset_index()['model'][i].score(X,y)
            RF_test.ix[i,j] = list(RF.values())[i].score(X,y)
            SVM_test.ix[i,j] = SVM.reset_index()['model'][i].score(X,y)
            LR_test.ix[i,j] = LR.reset_index()['model'][i].score(X,y)
            print("DNN_test" + str(j) + " : " + str(DNN_test.ix[i,j]))
            print("RF_test" + str(j) + " : " + str(RF_test.ix[i,j]))
            print("SVM_test" + str(j) + " : " + str(SVM_test.ix[i,j]))
            print("LR_test" + str(j) + " : " + str(LR_test.ix[i,j]))
            print("mean_ensemble_test" + str(j) + " : " + str(mean_ensemble_test.ix[i,j]))
            print("ensemble_test" + str(j) + " : " + str(ensemble_test.ix[i,j]))
            #precsion
            DNN_test2.ix[i,j] = precision_score(y, DNN_pred)
            RF_test2.ix[i,j] =  precision_score(y, RF_pred)
            SVM_test2.ix[i,j] = precision_score(y, SVM_pred)
            LR_test2.ix[i,j] =  precision_score(y, LR_pred)
            #recall
            DNN_test3.ix[i,j] = recall_score(y, DNN_pred)
            RF_test3.ix[i,j] =  recall_score(y, RF_pred)
            SVM_test3.ix[i,j] = recall_score(y, SVM_pred)
            LR_test3.ix[i,j] =  recall_score(y, LR_pred)
            #print("DNN_test" + str(j) + " : " + str(DNN_test2.ix[i,j]))
            #print("RF_test" + str(j) + " : " + str(RF_test2.ix[i,j]))
            #print("SVM_test" + str(j) + " : " + str(SVM_test2.ix[i,j]))
            #print("LR_test" + str(j) + " : " + str(LR_test2.ix[i,j]))
            
            
            #print("mean_ensemble_test" + str(j) + " : " + str(mean_ensemble_test.ix[i,j]))
            #print("ensemble_test" + str(j) + " : " + str(ensemble_test.ix[i,j]))
            
            
            
            s=s+1 #乱数調整
        else: #testに頻出部品iがなかったら計算しない。
            DNN_test.ix[i,j] = None
            RF_test.ix[i,j] = None
            SVM_test.ix[i,j] = None
            LR_test.ix[i,j] = None
            mean_ensemble_test.ix[i,j] = None
            ensemble_test.ix[i,j] = None                            
            DNN_test2.ix[i,j] = None
            RF_test2.ix[i,j] = None
            SVM_test2.ix[i,j] = None
            LR_test2.ix[i,j] = None
            mean_ensemble_test2.ix[i,j] = None
            ensemble_test2.ix[i,j] = None
            DNN_test3.ix[i,j] = None
            RF_test3.ix[i,j] = None
            SVM_test3.ix[i,j] = None
            LR_test3.ix[i,j] = None
            mean_ensemble_test3.ix[i,j] = None
            ensemble_test3.ix[i,j] = None
#モデルの学習時間
#print("train_time_DNN: " + str(sum(all_time_DNN_train)))
#print("train_time_RF: " + str(sum(all_time_RF_train)))
#print("train_time_SVM: " + str(sum(all_time_SVM_train)))
#print("train_time_LR: " + str(sum(all_time_LR_train)))
#print("train_time_ensemble: " + str(sum(all_time_DNN_train) + sum(all_time_RF_train) + sum(all_time_SVM_train) + sum(all_time_LR_train)))


db = db.rename(columns={"buhin":"部品品番"})


DB = db.copy()#pd.read_csv("DB.csv", encoding="shift_jis")
DB.index = DB["部品品番"]
DNN_train_validation = DNN_train_validation.loc[DB["部品品番"],:]
RF_train_validation = RF_train_validation.loc[DB["部品品番"],:]
SVM_train_validation = SVM_train_validation.loc[DB["部品品番"],:]
LR_train_validation = LR_train_validation.loc[DB["部品品番"],:]
mean_ensemble_train_validation = mean_ensemble_train_validation.loc[DB["部品品番"],:]
ensemble_train_validation = ensemble_train_validation.loc[DB["部品品番"],:]
#accuracy
DNN_test = DNN_test.loc[DB["部品品番"],:]
RF_test = RF_test.loc[DB["部品品番"],:]
SVM_test = SVM_test.loc[DB["部品品番"],:]
LR_test = LR_test.loc[DB["部品品番"],:]
mean_ensemble_test = mean_ensemble_test.loc[DB["部品品番"],:]
ensemble_test = ensemble_test.loc[DB["部品品番"],:]
#precision
DNN_test2 = DNN_test2.loc[DB["部品品番"],:]
RF_test2 = RF_test2.loc[DB["部品品番"],:]
SVM_test2 = SVM_test2.loc[DB["部品品番"],:]
LR_test2 = LR_test2.loc[DB["部品品番"],:]
mean_ensemble_test2 = mean_ensemble_test2.loc[DB["部品品番"],:]
ensemble_test2 = ensemble_test2.loc[DB["部品品番"],:]
#recall
DNN_test3 = DNN_test3.loc[DB["部品品番"],:]
RF_test3 = RF_test3.loc[DB["部品品番"],:]
SVM_test3 = SVM_test3.loc[DB["部品品番"],:]
LR_test3 = LR_test3.loc[DB["部品品番"],:]
mean_ensemble_test3 = mean_ensemble_test3.loc[DB["部品品番"],:]
ensemble_test3 = ensemble_test3.loc[DB["部品品番"],:]
                                           
#testデータでの各モデルの10回の平均accuracy   
mean_test_DNN = np.nanmean(DNN_test, axis=1) 
mean_test_RF = np.nanmean(RF_test, axis=1)
mean_test_SVM = np.nanmean(SVM_test, axis=1)
mean_test_LR = np.nanmean(LR_test, axis=1)
mean_test_mean_ensemble = np.nanmean(mean_ensemble_test, axis=1)
mean_test_ensemble = np.nanmean(ensemble_test, axis=1)
#testデータでの各モデルの10回の平均precsion  
mean_test_DNN2 = np.nanmean(DNN_test2, axis=1) 
mean_test_RF2 = np.nanmean(RF_test2, axis=1)
mean_test_SVM2 = np.nanmean(SVM_test2, axis=1)
mean_test_LR2 = np.nanmean(LR_test2, axis=1)
mean_test_mean_ensemble2 = np.nanmean(mean_ensemble_test2, axis=1)
mean_test_ensemble2 = np.nanmean(ensemble_test2, axis=1)
#testデータでの各モデルの10回の平均recall
mean_test_DNN3 = np.nanmean(DNN_test3, axis=1) 
mean_test_RF3 = np.nanmean(RF_test3, axis=1)
mean_test_SVM3 = np.nanmean(SVM_test3, axis=1)
mean_test_LR3 = np.nanmean(LR_test3, axis=1)
mean_test_mean_ensemble3 = np.nanmean(mean_ensemble_test3, axis=1)
mean_test_ensemble3 = np.nanmean(ensemble_test3, axis=1)

#精度に関してプリントする　
print("平均精度")
print("mean_acc_DNN_train_validation : " + str(np.nanmean(DNN_train_validation))) #train_validationでの300モデルのDNN平均accuracy
print("mean_acc_RF_train_validation : " + str(np.nanmean(RF_train_validation))) #train_validationでの300モデルのRF平均accuracy
print("mean_acc_SVM_train_validation : " + str(np.nanmean(SVM_train_validation))) #train_validationでの300モデルのSVM平均accuracy
print("mean_acc_LR_train_validation : " + str(np.nanmean(LR_train_validation))) #train_validationでの300モデルのLR平均accuracy
print("mean_acc_mean_ensemble_train_validation : " + str(np.nanmean(mean_ensemble_train_validation))) #train_validationでの300モデルのmean_ensemble平均accuracy
print("mean_acc_ensemble_train_validation : " + str(np.nanmean(ensemble_train_validation))) #train_validationでの300モデルのensemble平均accuracy
#accuracy
print("mean_acc_DNN_test : " + str(np.nanmean(mean_test_DNN))) #testでの300モデルのDNN平均accuracy
print("mean_acc_RF_test : " + str(np.nanmean(mean_test_RF))) #testでの300モデルのRF平均accuracy
print("mean_acc_SVM_test : " + str(np.nanmean(mean_test_SVM))) #testでの300モデルのSVM平均accuracy
print("mean_acc_LR_test : " + str(np.nanmean(mean_test_LR))) #testでの300モデルのLR平均accuracy
print("mean_acc_mean_ensemble_test : " + str(np.nanmean(mean_test_mean_ensemble))) #testでの300モデルのmean_ensemble平均accuracy
print("mean_acc_ensemble_test : " + str(np.nanmean(mean_test_ensemble))) #testでの300モデルのensemble平均accuracy
#precision
print("mean_acc_DNN_test : " + str(np.nanmean(mean_test_DNN2))) #testでの300モデルのDNN平均accuracy
print("mean_acc_RF_test : " + str(np.nanmean(mean_test_RF2))) #testでの300モデルのRF平均accuracy
print("mean_acc_SVM_test : " + str(np.nanmean(mean_test_SVM2))) #testでの300モデルのSVM平均accuracy
print("mean_acc_LR_test : " + str(np.nanmean(mean_test_LR2))) #testでの300モデルのLR平均accuracy
print("mean_acc_mean_ensemble_test : " + str(np.nanmean(mean_test_mean_ensemble2))) #testでの300モデルのmean_ensemble平均accuracy
print("mean_acc_ensemble_test : " + str(np.nanmean(mean_test_ensemble2))) #testでの300モデルのensemble平均accuracy
#recall
print("mean_acc_DNN_test : " + str(np.nanmean(mean_test_DNN3))) #testでの300モデルのDNN平均accuracy
print("mean_acc_RF_test : " + str(np.nanmean(mean_test_RF3))) #testでの300モデルのRF平均accuracy
print("mean_acc_SVM_test : " + str(np.nanmean(mean_test_SVM3))) #testでの300モデルのSVM平均accuracy
print("mean_acc_LR_test : " + str(np.nanmean(mean_test_LR3))) #testでの300モデルのLR平均accuracy
print("mean_acc_mean_ensemble_test : " + str(np.nanmean(mean_test_mean_ensemble3))) #testでの300モデルのmean_ensemble平均accuracy
print("mean_acc_ensemble_test : " + str(np.nanmean(mean_test_ensemble3))) #testでの300モデルのensemble平均accuracy

#受付NOを絞る(学習させた部品のみで構成される受付NO)
#部品が含まれていない受付NO
no_no = df2[~(df2["部品品番"].isin(buhin2))].受付NO.unique()
df3 = df2[~(df2["受付NO"].isin(no_no))]
#予測して持っていく部品の組み合わせを決める
V = df3.copy().drop_duplicates(['受付NO']).sort_values(by=["受付NO"]) #testデータを受付NOがuniqueになるように抽出
NO = sorted(list(set(V.loc[:,"受付NO"]))) #testデータの受付NOの集合
repair = pd.DataFrame(0, columns=["ensemble","mean_ensemble"], index=NO) #一発修理できたかを記録

CB = dict() #受付Noに対しての正解部品集合を格納
PB = dict() #受付Noに対しての多数決モデルでの予測部品集合を格納
PB_mean = dict() #受付Noに対しての平均モデルでの予測部品集合を格納
Probability_mean = pd.DataFrame(0, columns=buhin2, index=NO) #平均モデルでの予測確率を格納
Probability = pd.DataFrame(0, columns=buhin2, index=NO) #多数決モデルでの予測確率を格納

X = V.iloc[:,3:397] #受付NOに対しての特徴量ベクトル

all_time_DNN_test = np.zeros(len(buhin2))
all_time_RF_test = np.zeros(len(buhin2))
all_time_SVM_test = np.zeros(len(buhin2))
all_time_LR_test = np.zeros(len(buhin2))
all_time_ensemble_test = np.zeros(len(buhin2))
all_time_mean_ensemble_test = np.zeros(len(buhin2))

repeat = 0
for j in buhin2: #対象部品のループ
    repeat += 1
    #作ったモデルにtestデータを受付NO単位で入れる
    start_time = time.time()
    pre1 = DNN.loc[j,"model"].predict_proba(X)[:,1]
    end_time = time.time()
    all_time_DNN_test[repeat-1] = end_time - start_time
    
    start_time = time.time()
    pre2 = RF[j].predict_proba(X)[:,1]
    end_time = time.time()
    all_time_RF_test[repeat-1] = end_time - start_time
    
    start_time = time.time()
    pre3 = SVM.loc[j,"model"].predict_proba(X)[:,1]
    end_time = time.time()
    all_time_SVM_test[repeat-1] = end_time - start_time
    
    start_time = time.time()
    pre4 = LR.loc[j,"model"].predict_proba(X)[:,1]
    end_time = time.time()
    all_time_LR_test[repeat-1] = end_time - start_time
    
    start_time = time.time()
    pre_mat = pd.DataFrame({"DNN":pre1,"RF":pre2,"SVM":pre3,"LR":pre4},
                            index=NO)
    pre_mat["count"] = (pre_mat >= 0.5).sum(axis=1)
    print(j)
    print(pre_mat)
    
    
    for k in NO:
        if pre_mat.loc[k,"count"] == 3: #0.5以上が3つあったら上位3つの平均値で予測
            Probability.loc[k,j] = np.nanmean(np.sort(np.array([pre_mat.loc[k,"DNN"],pre_mat.loc[k,"RF"],pre_mat.loc[k,"SVM"],pre_mat.loc[k,"LR"]]))[1:])
        elif (pre_mat.loc[k,"count"] == 4) or (pre_mat.loc[k,"count"] == 0) or (pre_mat.loc[k,"count"] == 2): #0.5以上が0,2,4だったら中央値で予測
            Probability.loc[k,j] = np.median(pre_mat.loc[k,"DNN":"LR"])
        else : #0.5が1つだったら下位3つの平均値で予測
            Probability.loc[k,j] = np.nanmean(np.sort(np.array([pre_mat.loc[k,"DNN"],pre_mat.loc[k,"RF"],pre_mat.loc[k,"SVM"],pre_mat.loc[k,"LR"]]))[0:3])
    end_time = time.time()
    all_time_ensemble_test[repeat-1] = end_time - start_time + all_time_DNN_test[repeat-1] + all_time_RF_test[repeat-1] + all_time_SVM_test[repeat-1] + all_time_LR_test[repeat-1]
    
    start_time = time.time()    
    Probability_mean.loc[:,j] = (pre1 + pre2 + pre3 + pre4)/4 #平均モデルは平均値で予測
    end_time = time.time()
    all_time_mean_ensemble_test[repeat-1] = end_time - start_time + all_time_DNN_test[repeat-1] + all_time_RF_test[repeat-1] + all_time_SVM_test[repeat-1] + all_time_LR_test[repeat-1]
    
#計算時間の算出
#print("test_time_DNN: " + str(sum(all_time_DNN_test)))
#print("test_time_RF: " + str(sum(all_time_RF_test)))
#print("test_time_SVM: " + str(sum(all_time_SVM_test)))
#print("test_time_LR: " + str(sum(all_time_LR_test)))
#print("test_time_ensemble: " + str(sum(all_time_ensemble_test)))
#print("test_time_mean_ensemble: " + str(sum(all_time_mean_ensemble_test)))
################################################################################################################

#受付NO毎の必要な部品を格納
for i in NO:
    CB[i] = list(set(df2.copy().loc[df2["受付NO"] == i,"部品品番"]))
    
repair[:] = 0
for i in NO:    
    a = Probability_mean.loc[i,:].sort_values()[(335-20):335]
    
    A = a[a >= 0.5].index #0.5を超えたら部品jを持っていく
#     print(a)
    if set(A) >= set(CB[i]): #正解部品集合が予測部品集合に含まれているかチェック。（現時点では頻出じゃない部品は絶対に選べない）
        repair.loc[i,"mean_ensemble"] = 1
    else:
        repair.loc[i,"mean_ensemble"] = 0

print("従来方法での一発修理率"+len(repair[repair.mean_ensemble==1])/ len(repair)*100)



  0%|                                                                                          | 0/335 [00:00<?, ?it/s]

DNN_train_validation : 0.9897959183673469
RF_train_validation : 1.0
SVM_train_validation : 0.9387755102040817
LR_train_validation : 0.9591836734693877
mean_ensemble_train_validation : 0.9897959183673469
ensemble_train_validation : 0.9897959183673469
repeat 1 end


  0%|▏                                                                               | 1/335 [00:13<1:12:29, 13.02s/it]

DNN_train_validation : 0.9890282131661442
RF_train_validation : 0.9937304075235109
SVM_train_validation : 0.9482758620689655
LR_train_validation : 0.9843260188087775
mean_ensemble_train_validation : 0.9858934169278997
ensemble_train_validation : 0.987460815047022
repeat 2 end


  1%|▍                                                                               | 2/335 [01:03<2:14:55, 24.31s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████▊| 334/335 [4:41:32<00:22, 22.69s/it]

DNN_train_validation : 1.0
RF_train_validation : 1.0
SVM_train_validation : 0.9736842105263158
LR_train_validation : 1.0
mean_ensemble_train_validation : 1.0
ensemble_train_validation : 1.0
repeat 335 end


100%|██████████████████████████████████████████████████████████████████████████████| 335/335 [4:41:43<00:00, 19.42s/it]


[0.92208533 0.72306751]
DNN_test0 : 0.5
RF_test0 : 1.0
SVM_test0 : 0.5
LR_test0 : 0.5
mean_ensemble_test0 : 1.0
ensemble_test0 : 1.0
[0.92208533 0.92208533]
DNN_test1 : 0.5
RF_test1 : 0.5
SVM_test1 : 0.5
LR_test1 : 0.5
mean_ensemble_test1 : 0.5
ensemble_test1 : 0.5
[0.92208533 0.03908301]
DNN_test2 : 1.0
RF_test2 : 1.0
SVM_test2 : 0.5
LR_test2 : 1.0
mean_ensemble_test2 : 1.0
ensemble_test2 : 1.0
[0.92208533 0.46319332]
DNN_test3 : 1.0
RF_test3 : 1.0
SVM_test3 : 0.5
LR_test3 : 0.5
mean_ensemble_test3 : 1.0
ensemble_test3 : 1.0
[0.92208533 0.92208533]
DNN_test4 : 0.5
RF_test4 : 0.5
SVM_test4 : 0.5
LR_test4 : 0.5
mean_ensemble_test4 : 0.5
ensemble_test4 : 0.5
[0.92208533 0.04006911]
DNN_test5 : 1.0
RF_test5 : 1.0
SVM_test5 : 0.5
LR_test5 : 1.0
mean_ensemble_test5 : 1.0
ensemble_test5 : 1.0
[0.92208533 0.91985623]
DNN_test6 : 0.5
RF_test6 : 0.5
SVM_test6 : 0.5
LR_test6 : 0.5
mean_ensemble_test6 : 0.5
ensemble_test6 : 0.5
[0.92208533 0.72503019]
DNN_test7 : 0.5
RF_test7 : 1.0
SVM_test7 : 0.

TypeError: can only concatenate str (not "float") to str

## 部品間の組み合わせを検討

予測確率が高い部品と一緒に持っていかれる回数が高い上位3部品を予測部品集合に追加する

In [49]:
repair[:] = 0
a_co_dic = dict()
a_dic = dict()
for i in NO:
    a_co = []
    a = Probability_mean.loc[i,:].sort_values()[(335-10):335]
    for j in range(10):
        b_three = df_recept[a.sort_values(ascending=False).index[j]].sort_values(ascending=False)[1:4]#ある部品の共起回数上位３つ
        for k in b_three.index:
            if a.index.isin([k]).any():
                pass
            else:
                if len(a_co)<10:
                    a_co.append(k)
                #print(a)
    a_2= a.index.tolist()
    a_co = list(set(a_co))
    a_co_dic[i] = a_co
    a_dic[i] = list(a.index)
    a_2.extend(a_co)
    A = a_2
    #print(len(A))
    #A = a[a >= 0.5].index #0.5を超えたら部品jを持っていく
#     print(a)
    if set(A) >= set(CB[i]): #正解部品集合が予測部品集合に含まれているかチェック。（現時点では頻出じゃない部品は絶対に選べない）
        repair.loc[i,"mean_ensemble"] = 1
    else:
        repair.loc[i,"mean_ensemble"] = 0

print("新手法での一発修理率"+len(repair[repair.mean_ensemble==1])/ len(repair)*100)
print("最終的に持っていく部品"+A)

83.35246842709529
